In [138]:
# Import libraries and modules
import numpy as np
import pandas as pd
import pprint

#from sklearn.model_selection import train_test_split
from sklearn import pipeline
from sklearn import preprocessing
from sklearn import feature_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
#from sklearn.metrics import f1_score
from sklearn.externals import joblib 

# Set random seed
#np.random.seed(222)

In [166]:
#Load the data.
data = pd.read_csv(filepath_or_buffer='./classify.csv', sep=' ', header=None)

In [140]:
#print data.head()
#print data.shape
#data.iloc[1,2]

In [163]:
# drop the last column
data.drop(labels=data.columns[len(data.columns)-1], axis=1, inplace=True)
#print data.head()

In [142]:
# Split data into training and test sets
y = data.iloc[:,0]
X = data.iloc[:, 1:len(data.columns)]
#X_train, X_test, y_train, y_test = train_test_split(X, y, 
#                                                   test_size=0.2, 
#                                                  random_state=222, 
#                                                    stratify=y)

In [143]:
## Make pipeline using data preprocessing step, best k-features selection step and classifier algo step
pipe = pipeline.Pipeline([('scaler', preprocessing.StandardScaler()), 
                         ('selectkbest', feature_selection.SelectKBest()),
                         ('randomforestclassifier', RandomForestClassifier(random_state=222))])

In [144]:
#print pipe.get_params()

# Random Forest Classifier without prunning:

In [145]:
## Declare default hyperparameters to tune i.e. giving the default values as it is
defaulat_hyperparam = {'selectkbest__k':[17]} # selecting all the features in the default no-prun run

In [146]:
# Create model without pruning using cross-validation pipeline
no_prune_clf = GridSearchCV(estimator=pipe, param_grid=defaulat_hyperparam, scoring="f1", cv=10)
no_prune_clf.fit(X, y)

GridSearchCV(cv=10, error_score='raise',
       estimator=Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('selectkbest', SelectKBest(k=10, score_func=<function f_classif at 0x7fa3edcd5758>)), ('randomforestclassifier', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_f...timators=10, n_jobs=1, oob_score=False, random_state=222,
            verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'selectkbest__k': [17]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score=True, scoring='f1', verbose=0)

In [147]:
## Refit on the entire training set
# No additional code needed if no_prune_clf.refit == True (default is True)

In [148]:
# Evaluate model pipeline on test data
#pred = no_prune_clf.predict(X_test)

## F1 score for Classifier without pruning:

In [164]:
features = no_prune_clf.best_estimator_.named_steps['selectkbest']
print "******** Random-Forest Classifier: No pruning: All features"
print "The feature selection scores(ANOVA F-value between label/feature) are as follows:"
print "Feature     Score "
for itr in range(X.shape[1]):
    print(str(itr+1) +"           "+str(features.scores_[itr]))

******** Random-Forest Classifier: No pruning: All features
The feature selection scores(ANOVA F-value between label/feature) are as follows:
Feature     Score 
1           7.23772367222
2           0.0149336154399
3           1.03811753159
4           0.000670197576397
5           2.46642335713
6           2.13170312392
7           0.116204644838
8           17.1518397529
9           16.5446655022
10           6.14741260717
11           0.265006093075
12           0.373368850308
13           0.0484537489148
14           0.00122740114148
15           0.0891489246438
16           1.029444709
17           0.647544904034


In [150]:
#print "F1 score for Classifier without pruning:", f1_score(y_test.values, pred, average=None)

In [151]:
print "******** Random-Forest Classifier: No pruning: All features: 10-fold Cross-Validation results: "
pprint.pprint(no_prune_clf.cv_results_)

******** Random-Forest Classifier: No pruning: All features: 10-fold Cross-Validation results: 
{'mean_fit_time': array([ 0.04338052]),
 'mean_score_time': array([ 0.0082346]),
 'mean_test_score': array([ 0.70272109]),
 'mean_train_score': array([ 0.99298246]),
 'param_selectkbest__k': masked_array(data = [17],
             mask = [False],
       fill_value = ?)
,
 'params': ({'selectkbest__k': 17},),
 'rank_test_score': array([1], dtype=int32),
 'split0_test_score': array([ 0.85714286]),
 'split0_train_score': array([ 1.]),
 'split1_test_score': array([ 0.5]),
 'split1_train_score': array([ 0.98245614]),
 'split2_test_score': array([ 0.66666667]),
 'split2_train_score': array([ 1.]),
 'split3_test_score': array([ 0.66666667]),
 'split3_train_score': array([ 1.]),
 'split4_test_score': array([ 0.85714286]),
 'split4_train_score': array([ 1.]),
 'split5_test_score': array([ 0.57142857]),
 'split5_train_score': array([ 0.98245614]),
 'split6_test_score': array([ 1.]),
 'split6_train_scor

In [152]:
print "******** Random-Forest Classifier: No pruning: All features: F1-score of the best estimator: ", no_prune_clf.best_score_

******** Random-Forest Classifier: No pruning: All features: F1-score of the best estimator:  0.702721088435


In [153]:
#Save model for future use
joblib.dump(no_prune_clf, 'no_prune_clf.pkl')
# To load: no_prune_clf = joblib.load('no_prune_clf.pkl')

['no_prune_clf.pkl']

# Random Forest Classifier with prunning:

In [154]:
## Declare hyperparameters for prunning
pruning_hyperparam = { 'selectkbest__k':[12]
                       #randomforestclassifier__n_estimators' : [10]
                       #,randomforestclassifier__max_features' : ['sqrt']
                        ,'randomforestclassifier__min_samples_leaf' : [2]
                        ,'randomforestclassifier__min_samples_split' : [4]
                     }

In [155]:
# Create model with pruning using cross-validation pipeline
pruned_clf = GridSearchCV(estimator=pipe, param_grid=pruning_hyperparam, scoring="f1", cv=10)
pruned_clf.fit(X, y)

GridSearchCV(cv=10, error_score='raise',
       estimator=Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('selectkbest', SelectKBest(k=10, score_func=<function f_classif at 0x7fa3edcd5758>)), ('randomforestclassifier', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_f...timators=10, n_jobs=1, oob_score=False, random_state=222,
            verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'randomforestclassifier__min_samples_leaf': [2], 'randomforestclassifier__min_samples_split': [4], 'selectkbest__k': [12]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='f1', verbose=0)

In [156]:
# Evaluate model pipeline on test data
#pruned_pred = pruned_clf.predict(X_test)

## F1 score for Classifier with pruning:

In [157]:
#print "F1 score for Classifier with pruning:", f1_score(y_test.values, pruned_pred, average=None)

In [158]:
print "******** Random-Forest Classifier: Pruning parameters are set as follows:"
print "Number of best features to be considered (out of 17): 12"
print "min_samples_leaf(The minimum number of samples required to be at a leaf node): 2 "
print "min_samples_split(The minimum number of samples required to split an internal node): 4 "

******** Random-Forest Classifier: Pruning parameters are set as follows:
Number of best features to be considered (out of 17): 12
min_samples_leaf(The minimum number of samples required to be at a leaf node): 2 
min_samples_split(The minimum number of samples required to split an internal node): 4 


In [159]:
print "******** Random-Forest Classifier: With pruning : 10-fold Cross-Validation results: "
pprint.pprint(pruned_clf.cv_results_)

******** Random-Forest Classifier: With pruning : 10-fold Cross-Validation results: 
{'mean_fit_time': array([ 0.04313991]),
 'mean_score_time': array([ 0.00827041]),
 'mean_test_score': array([ 0.69930736]),
 'mean_train_score': array([ 0.94623529]),
 'param_randomforestclassifier__min_samples_leaf': masked_array(data = [2],
             mask = [False],
       fill_value = ?)
,
 'param_randomforestclassifier__min_samples_split': masked_array(data = [4],
             mask = [False],
       fill_value = ?)
,
 'param_selectkbest__k': masked_array(data = [12],
             mask = [False],
       fill_value = ?)
,
 'params': ({'randomforestclassifier__min_samples_leaf': 2,
             'randomforestclassifier__min_samples_split': 4,
             'selectkbest__k': 12},),
 'rank_test_score': array([1], dtype=int32),
 'split0_test_score': array([ 0.75]),
 'split0_train_score': array([ 0.94736842]),
 'split1_test_score': array([ 0.72727273]),
 'split1_train_score': array([ 0.93103448]),
 'spli

In [160]:
#pruned_clf.best_params_

In [161]:
print "******** Random-Forest Classifier: With pruning : F1-score of the best estimator: ", pruned_clf.best_score_

******** Random-Forest Classifier: With pruning : F1-score of the best estimator:  0.699307359307


In [162]:
#Save model for future use
joblib.dump(pruned_clf, 'pruned_clf.pkl')
# To load: pruned_clf = joblib.load('pruned_clf.pkl')

['pruned_clf.pkl']